<a href="https://colab.research.google.com/github/jongjunkim/Prediction-of-Stock-Market/blob/main/CS435_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/CS435_Project/

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/CS435_Project/'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import copy
import yfinance as yf
import math


In [ ]:
with open('/content/gdrive/MyDrive/CS435_Project/training_set.pkl', 'rb') as f:
        data = pickle.load(f)

with open('/content/gdrive/MyDrive/CS435_Project/training_set.pkl', 'rb') as t1:
        test1 = pickle.load(t1)

with open('/content/gdrive/MyDrive/CS435_Project/training_set.pkl', 'rb') as t2:
        test2 = pickle.load(t2)

original_data = data
#copy from original data so that it doesn't modify the original data
data_frame = copy.deepcopy(original_data)

#Task1

In [ ]:
#Calculate the daily percentage changes of the close prices for each stock and return the daily percentage changes 
def daily_percentage_change_for_close(data):
    percentage_changes = []
    for df in data:
        close_prices = df['Close']
        pct_changes = []
        for i in range(1, len(close_prices)):
            today_close = close_prices[i-1]
            next_close = close_prices[i]
            pct_change = ((next_close - today_close) / today_close) * 100
            pct_changes.append(pct_change)
        percentage_changes.append(pct_changes)
    return percentage_changes
    

In [ ]:
def divide_into_levels(percentage_changes, thresholds):
    """
    Divides the daily percentage changes into three levels based on the provided thresholds.

    Parameters:
    percentage_changes (list of pandas.Series): The daily percentage changes for each stock.
    thresholds (list of tuples): The two threshold values.

    Returns:
    list of lists: The level classifications for each day of each stock.
    """
    levels = []
    for i in range(len(percentage_changes)):
        stock_levels = []
        for j in range(len(percentage_changes[i])):
            if percentage_changes[i][j] < thresholds[0][0]:
                stock_levels.append(0)
            elif percentage_changes[i][j] > thresholds[0][1]:
                stock_levels.append(2)
            else:
                stock_levels.append(1)
        levels.append(stock_levels)
    return levels

In [ ]:
def create_label_vectors(levels, labels):
    """
    Converts levels into a 3-dimensional vector representation.

    Parameters:
    levels (list of lists): The levels for each stock.
    labels (list of str): The labels corresponding to each level.

    Returns:
    numpy.ndarray: The label vectors.
    """
    num_classes = len(labels)

    # Convert the levels to integer values
    label_integers = []
    for stock_levels in levels:
        stock_integers = []
        for level in stock_levels:
            if level == 0:
                stock_integers.append(0)
            elif level == 1:
                stock_integers.append(1)
            elif level == 2:
                stock_integers.append(2)
        label_integers.append(stock_integers)

    # Convert the integer labels to label vectors
    label_vectors = np.zeros((len(label_integers), len(label_integers[0]), num_classes))
    for i in range(len(label_integers)):
        for j in range(len(label_integers[i])):
            label_vectors[i, j, label_integers[i][j]] = 1

    return label_vectors

In [ ]:
#print the two threshold used to divde data into three levels
#add the number of data points for each level and print them
def threshold_number_of_data_points(levels, thresholds):

    increase = 0
    decrease = 0
    no_change = 0

    for i in range(len(levels)):
        increase += sum([1 for level in levels[i] if level == 2])
        decrease += sum([1 for level in levels[i] if level == 0])
        no_change += sum([1 for level in levels[i] if level == 1])

    print("Threshold 1:", thresholds[0][0])
    print("Threshold 2:", thresholds[0][1])
    print("Number of increases:", increase)
    print("Number of decreases:", decrease)
    print("Number of no big changes:", no_change)

In [ ]:
#To get the two threshold, we use 33th and 66th percentile.
#return list of thresholds   
def find_thresholds(closed_percentage):

    #combine 2000X2202 data int one dimesional data
    combine_into_one = np.array(closed_percentage).ravel()
    sorted_closed_percentage = np.sort(combine_into_one)
    thresholds = []
    threshold1 = np.nanpercentile(sorted_closed_percentage, 33.3)
    threshold2 = np.nanpercentile(sorted_closed_percentage, 66.6)
    thresholds.append((threshold1, threshold2))

    return thresholds

#Task2

In [ ]:
#calculate a daily return which is calculated by substracting the opening price from the closing price
def daily_return(data_frame):
    for i, df in enumerate(data_frame):
        data_frame[i]['Daily_return'] = df.Close - df.Open

In [ ]:
#daily log return
def daily_log_return(data_frame):
    
    for i, df in enumerate(data_frame):
        data_frame[i]['log_return'] = np.log(df['Close'] / df['Close'].shift(1))

In [ ]:
def moving_average(data_frame, window_size):
      
    for i, df in enumerate(data_frame):
        close_prices = df['Close']
        moving_average = close_prices.rolling(window = window_size).mean()
        data_frame[i][str(window_size) + '-day MA'] = moving_average

In [ ]:
#calculate standard_devation
def standard_deviation(data_frame, window_size):

    for i, df in enumerate(data_frame):
        close_prices = df['Close']
        std_dev = close_prices.rolling(window =window_size).std()
        data_frame[i][str(window_size) + '-day S.D'] = std_dev

In [ ]:
def median_price(data_frame):

    for i,df in enumerate(data_frame):
        data_frame[i]['median'] = (df.High + df.Low)/2

In [ ]:
#calcualte volatility
def volatility(data_frame, window_size = 10):

    for i,df in enumerate(data_frame):
        data_frame[i]['volatility'] = abs(df['Close'] - df['Close'].shift(1)).rolling(window_size).sum()

In [ ]:
#Donchian_Channel consists of Lower Channel, Upper Channel, and Middle Channel
#typically 20 days
def donchian_channel(data_frame, window_size = 20):

    for i, df in enumerate(data_frame):
        data_frame[i]['Higher_Channel'] = df.High.rolling(window_size).max()
        data_frame[i]['Lower_Channel'] = df.Low.rolling(window_size).min()
        data_frame[i]['Middle_Channel'] = (df.Higher_Channel + df.Lower_Channel) / 2

In [ ]:

#calculates upper and lower Bollinger Bands, using moving_averge and two standard deviation
#upperband and lower band are the lines that are plotted at a distance of 'k'
#typically window_size is 20 and k is usually 2
#added Bollinger Bands Squeeze - BandWidth, Avg BandWidth, Low Volatility, Breakout, and Breakdown
def bollinger_bands(data_frame, window_size, k = 2):

    for i,df in enumerate(data_frame):
        close_prices = df['Close']
        moving_average = close_prices.rolling(window_size).mean()
        std_dev = close_prices.rolling(window_size).std()
        upper_band = moving_average + k * std_dev
        lower_band = moving_average - k * std_dev
        data_frame[i][str(window_size) + '-day BOLU'] = upper_band
        data_frame[i][str(window_size) + '-day BOLD'] = lower_band

In [ ]:
#Expoenetial Moving Average that gives more weight to recent prices 
#short term tpyically (12 or 26 days) while Long term (50 day or 200 days) 
def ema(data_frame, window_size):

    for i,df in enumerate(data_frame):
        exponetial = df.Close.ewm(span=window_size, adjust = False).mean()
        data_frame[i][str(window_size) + '-day EMA'] = exponetial

In [ ]:
#Zero Lag Exponential Moving average(ZLEMA)
def zero_lag_ema(data_frame, window_size = 5):

    for i, df in enumerate(data_frame):
        lag = int((window_size-1)/2)
        emadata = df.Close + (df.Close - df.Close.shift(lag))
        data_frame[i]['ZLEMA'] = emadata.ewm(span = window_size, adjust = False).mean()

In [ ]:
# Calculate weighted_moving_average that assigns higher weight to more recent values and lower weight to older values
# 90 period weighted_moving_average
def WMA(data_frames, window_size = 14):

    for i, df in enumerate(data_frames):
        w = np.arange(1, window_size + 1, 1)
        wma = df.Close.rolling(window_size).apply(lambda x: np.dot(x, w)/w.sum(), raw=True)
        data_frames[i][str(window_size) + '-day WMA'] = wma

In [ ]:

#Hull Moving Average(MMA)
def Hull_MA(data_frame, window_size = 29):


    def get_wma(data, period):
        w = np.arange(1, window_size + 1, 1)
        wma = df.Close.rolling(window_size).apply(lambda x: np.dot(x, w)/w.sum(), raw=True)
        return wma
    
    for i,df in enumerate(data_frame):
        wma1 = get_wma(df, window_size/2)
        wma2 = get_wma(df, window_size)
        raw_hma = (2 * wma1) - wma2
        data_frame[i]['HMA'] = get_wma(raw_hma, math.sqrt(window_size))

In [ ]:
# Calculate Momentum that measures the strength of the price trend in an asset 
# typical momentum day is 14 days or 21 days
def momentum(data_frames, window_size = 14):
    
    for i,df in enumerate(data_frames):
        stock_momentum = df.Close.diff(window_size)
        data_frames[i][str(window_size) + '-day Momentum'] = stock_momentum

In [ ]:
#Rate of Change
#Measures the percentage change the current closing price and the closing price n-time days ago and ange
#Standard calculation is 12, 25, 200
def ROC(data_frames, window_size):
 
    for i,df in enumerate(data_frames):
        stock_roc = ((df.Close - df.Close.shift(window_size)) / df.Close.shift(window_size) * 100)
        data_frames[i][str(window_size) + '-day ROC'] = stock_roc

In [ ]:
#Relative Strength Index = Dynamic Momentum Index(DMI)
#Momentum indicator that compares the magnitude of recent gains to recent losses to determine overbought or oversold conditions in an asset.
# typcially window size is 14 days
def RSI(data_frames, window_size = 14):
    
   for i, df in enumerate(data_frames):
        close_prices = df['Close']
        diff = close_prices.diff()
        up, down = diff.copy(), diff.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        gain = up.rolling(window_size).mean()
        loss = down.abs().rolling(window_size).mean()
        rs = gain / loss
        rsi = 100.0 - (100.0 / (1.0 + rs))
        data_frames[i]['RSI'] = rsi

In [ ]:
#Stochastic Oscillator
#momentum indicator that compares a security's close price to its price range over a specified number of days
#if below 20% oversold, above 80% overbought
#typically 14-day day
def stochastic_oscillator(data_frames, window_size = 14):

    for i,df in enumerate(data_frames):
        close = df['Close']
        low = df['Low'].rolling(window_size).min()
        high = df['High'].rolling(window_size).max()
        stochastic = (close - low) / (high - low) * 100
        data_frames[i][str(window_size) + '-day Stoch'] = stochastic
        slowStoch = stochastic.rolling(3).mean()
        data_frames[i]['3_day Stoch'] = slowStoch

In [ ]:
#Awesome Oscillator(AO)
#shows what's happening to the market driving force at the present moment
def awesome_Oscillator(data_frames, short_period = 5, long_period = 34):

    for i, df in enumerate(data_frames):
        median_price = (df.High + df.Low)/2
        data_frames[i]['AO'] = median_price.rolling(short_period).mean() - median_price.rolling(long_period).mean()

In [ ]:
#Calculate Moving Average Convergence Divergence
def MACD(data_frames, fast_day=12, slow_day=26, signal_day=9):
    for i, df in enumerate(data_frames):
        fast_ma = df.Close.ewm(span = fast_day, adjust = False).mean()
        slow_ma = df.Close.ewm(span = slow_day, adjust = False).mean()
        data_frames[i]['MACD'] = fast_ma - slow_ma
        signal_line = df.MACD.ewm(span=signal_day, adjust = False).mean()
        data_frames[i]['signal Line'] = signal_line

In [ ]:
#Williams %R  
#momentum oscillator that measures overbought and oversold levels 
#typically 14 days
def williams_R(data_frames, lookback_day = 14):

    for i, df in enumerate(data_frames):
        highest_high = df['High'].rolling(lookback_day).max()
        lowest_low = df['Low'].rolling(lookback_day).min()
        williams_R = (highest_high - df['Close']) / (highest_high - lowest_low) * -100
        data_frames[i][str(lookback_day) + '-day William %R'] = williams_R

In [ ]:
#Aroon indictor
#determine if an stock is in a trend and how strong the trend is
def aroon_indicator(data_frames, window_size = 25):

    for i, df in enumerate(data_frames):
        aroon_high = df['High'].rolling(min_periods = window_size, window = window_size, center = False).apply(np.argmax)
        aroon_low = df['Low'].rolling(min_periods = window_size, window = window_size, center = False).apply(np.argmin)
        aroon_up = (((window_size - (aroon_high + 1)) / window_size)) * 100 
        aroon_down = (((window_size - (aroon_low + 1)) / window_size)) * 100 
        data_frames[i][str(window_size) + '-day Aroon Up'] = aroon_up
        data_frames[i][str(window_size) + '-day Aroon Down'] = aroon_down
        data_frames[i]['Aroon Oscillator'] = aroon_up - aroon_down

In [ ]:
#Pivot points (pivot point, Resistance1, Resistance2, Support1, Support2)
def pivot_points(data_frame):

    for i,df in enumerate(data_frame):
        data_frame[i]['pivot_point'] = (df['High'] + df['Low'] + df['Close']) / 3
        data_frame[i]['Resistance1'] = (df.pivot_point * 2) - df.Low
        data_frame[i]['Resistance2'] = (df.pivot_point + (df.High - df.Low))
        data_frame[i]['Support1'] = (df.pivot_point * 2) - df.High
        data_frame[i]['Supprot2'] = (df.pivot_point -  (df.High - df.Low))

In [ ]:
#Commodity Channel Index
#relatively high when prices are far from average
def CCI(data_frame, window_size = 20):
    
    for i, df in enumerate(data_frame):
        typical_price = (df['High'] + df['Low'] + df['Close']) / 3
        tp_avg = typical_price.rolling(window_size).mean()
        mean_deviation =  abs(typical_price - tp_avg).rolling(window_size).mean()
        cci = (typical_price - tp_avg) / (0.015 * mean_deviation)
        data_frame[i][str(window_size)+ '-day CCI'] = cci

In [ ]:
#Coppock curve to determine if it is bull market
#Typciall applied roc14 is 14period(months) = 420 and roc11 is 11 period(months) = 330
#10 months of weighted average
def coppock_curve(data_frame, wma_period = 300, roc14_period = 420, roc11_period = 330):

    def get_wma(data, window_size):
        w = np.arange(1, window_size + 1, 1)
        wma = data.rolling(window_size).apply(lambda x: np.dot(x, w)/w.sum(), raw=True)
        return wma
    
    def get_roc(data, window_size):
        stock_roc = ((data - data.shift(window_size)) / data.shift(window_size) * 100)
        return stock_roc

    for i, df in enumerate(data_frame):
        long_roc = get_roc(df.Close, roc14_period)
        short_roc = get_roc(df.Close, roc11_period)
        ROC = long_roc + short_roc
        coppock = get_wma(ROC, wma_period)
        data_frame[i]['Coppock Curve'] = coppock

In [ ]:
#On balance Volume(OBV) and On_balance_Volume_Mean
def on_balance_volume(data_frame):
    
    for i, df in enumerate(data_frame):
        data_frame[i]['OBV'] = (np.sign(df["Close"].diff()) * df["Volume"]).cumsum()
        data_frame[i]['OBV_Mean'] = df.OBV.rolling(14).mean()

In [ ]:
#Volume Oscillator(VO)
def Volume_Oscillator(data_frame, short_window = 14, long_window =28):

    for i,df in enumerate(data_frame):
        short_period = df.Volume.rolling(short_window).mean()
        long_period = df.Volume.rolling(long_window).mean()
        data_frame[i]['VO'] = ((short_period - long_period)/long_period) * 100

In [ ]:
#Chaikin Money Flow(CMF) (MF = money flow)
#typically 20 period
def chaikin_money_flow(data_frame):

    for i, df in enumerate(data_frame):
        data_frame[i]['MF_Multiplier'] = ((df.Close - df.Low) - (df.High - df.Close)) / (df.High - df.Low)
        data_frame[i]['MF_Volume'] = df.MF_Multiplier * df.Volume
        data_frame[i]['CMF'] = df.MF_Volume.rolling(21).mean() / df.Volume.rolling(21).mean()

In [ ]:
#accumulation_distribution(ADI)
#use relationship between the stock's price and volume flow
def accumulation_distribution(data_frame):

    for i, df in enumerate(data_frame):
        data_frame[i]['ADI'] = df.MF_Volume.cumsum()

In [ ]:
#Fibonacci retracement levels(FRL)
#The Fibanoacci Ratios are typically 0.236 0.382 0.618 
def Fibonacci_retracement_levels(data_frame):

    for i, df in enumerate(data_frame):
        maximum = df.Close.max()
        minimum = df.Close.min()
        diff = maximum - minimum
        data_frame[i]['FRL_lev1'] = maximum - 0.236 * diff
        data_frame[i]['FRL_lev2'] = maximum - 0.382 * diff
        data_frame[i]['FRL_lev3'] = maximum - 0.618 * diff

In [ ]:
#Money Flow Index(MFI)
#MFI above 80 considred overbought and below 20 oversold
def money_flow_index(data_frame, period = 14):

    def get_mfi(df, period = 14):
        typical_price = (df['Close'] + df['High'] + df['Low']) / 3
        money_flow = typical_price * df['Volume']
        positive_flow = [money_flow[i-1] if typical_price[i] > typical_price[i-1] else 0 for i in range(1, len(typical_price))]
        negative_flow = [money_flow[i-1] if typical_price[i] < typical_price[i-1] else 0 for i in range(1, len(typical_price))]
        positive_mf = [sum(positive_flow[i + 1- period : i+1]) for i in range(period-1, len(positive_flow))]
        negative_mf = [sum(negative_flow[i + 1- period : i+1]) for i in range(period-1, len(negative_flow))]
        MFI = 100 * (np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf)))
        df.loc[period:, 'MF'] = money_flow
        df.loc[period:, 'MFI'] = MFI

    for i, df in enumerate(data_frame):
        get_mfi(df)

In [ ]:
#the Triple Exponential Moving Average(TEMA or sometimes called TRIX)
def TEMA(data_frame, window_size = 15):

    for i, df in enumerate(data_frame):
        exponetial1 = df.Close.ewm(span=window_size, adjust = False).mean()
        exponetial2 = exponetial1.ewm(span = window_size, adjust = False).mean()
        exponetial3 = exponetial1.ewm(span = window_size, adjust = False).mean()
        data_frame[i]['TEMA'] = (3*exponetial1) - (3*exponetial2) + exponetial3

In [ ]:
#Volume Price Trend indicator, sometimes known as the Price_Volume period
#also called Volume Price Trend
def VPT(data_frame):

    for i, df in enumerate(data_frame):
        close_diff = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)
        vpt = df['Volume'] * close_diff
        vpt = vpt.cumsum()
        data_frame[i]['VPT'] = vpt

In [ ]:
#Average Daily Trading Volume
#typcially 20days or 30days
def ADTV(data_frame, window_size = 30):

    for i,df in enumerate(data_frame):
        data_frame[i][str(window_size) + 'day ADTV'] = df.Volume.rolling(window_size).sum()/window_size

In [ ]:
#Volume Weighted Average Price
def VWAP(data_frame):

    for i, df in enumerate(data_frame):
        cumulative_tp = df.pivot_point.cumsum()
        cumulative_volume = df.Volume.cumsum()
        data_frame[i]['VWAP'] = cumulative_tp/cumulative_volume

In [ ]:
#ICHIMOKU Cloud Trading
#Tenkan_sen = Conversion Line, Kijun_sen = Base line, Senkou span= Leading Span 
#Ichimoku cloud is the area where located between Senkou span1 and span2
def ichimoku_cloud(data_frame):

    for i, df in enumerate(data_frame):
        high_9 = df.High.rolling(window = 9).max()
        low_9 = df.Low.rolling(window = 9).min()
        high_26 = df.High.rolling(window = 26).max()
        low_26 = df.Low.rolling(window = 26).min()
        high_52 = df.High.rolling(window = 52).max()
        low_52 = df.Low.rolling(window = 52).min()
        data_frame[i]['Tenkan_sen'] = (high_9 + low_9)/2
        data_frame[i]['Kijun_sen'] = (high_26 + low_26)/2
        data_frame[i]['Senkou_span1'] = ((df.Tenkan_sen + df.Kijun_sen) / 2).shift(26)
        data_frame[i]['Senkou_span2'] = ((high_52 + low_52) / 2).shift(26)

In [ ]:
#Disparity index
#measures the relative position of an aseet's most recent closing price
#value greater than zero- asset is gaining upward momentum, value less than zero- a sign that selling pressure is increasing
def disparity(data_frame, window_size = 14):

    for i, df in enumerate(data_frame):
        mv = df.Close.rolling(window_size).mean()
        data_frame[i]['Disparity'] = (df.Close - mv)/(mv * 100)

In [ ]:
#detrended price oscillaotor
def DPO(data_frame, window_size = 20):

    for i, df in enumerate(data_frame):
        detrended_period = int(window_size/2 + 1)
        data_frame[i]['DPO'] = df['Close'].shift(detrended_period)  - df['Close'].rolling(window_size).mean()

In [ ]:
#Ease of Movement
def EMV(data_frame, window_size = 14):

    for i,df in enumerate(data_frame):
        distance_moved = (df.High + df.Low)/2 - (df.High.shift(1) + df.Low.shift(1))/2
        box_ratio = (df.Volume/1000000) / (df.High - df.Low)
        emv = distance_moved/box_ratio
        data_frame[i][str(window_size) + 'day EMV'] = emv.rolling(window_size).mean()

In [ ]:

#Force Index(FI)
def ForceIndex(data_frame, window_size = 14):

    for i,df in enumerate(data_frame):
        forceindex = (df.Close - df.Close.shift(1)) * df.Volume
        data_frame[i]['1day_FI'] = forceindex
        data_frame[i][str(window_size) + 'day FI'] = forceindex.ewm(span = window_size, adjust = False).mean()

In [ ]:
#Keltner channel(KC)
def Keltner_channel(data_frame, window_size = 14):

    for i,df in enumerate(data_frame):
        data_frame[i]['KC_middle'] = df.Close.ewm(span = window_size, adjust = False).mean()
        data_frame[i]['KC_upper'] = df.KC_middle + df.MF_Multiplier * df.ATR
        data_frame[i]['KC_lower'] = df.KC_middle - df.MF_Multiplier * df.ATR

In [ ]:
#Know Sure Thing(KST)
def Know_sure_thing(data_frame):

    for i,df in enumerate(data_frame):
        ROC_10 = ((df.Close - df.Close.shift(10)) / df.Close.shift(10) * 100)
        ROC_15 = ((df.Close - df.Close.shift(15)) / df.Close.shift(15) * 100)
        ROC_20 = ((df.Close - df.Close.shift(20)) / df.Close.shift(20) * 100)
        ROC_30 = ((df.Close - df.Close.shift(30)) / df.Close.shift(30) * 100)
        RCMA1 = ROC_10.rolling(10).mean()
        RCMA2 = ROC_15.rolling(10).mean()
        RCMA3 = ROC_20.rolling(10).mean()
        RCMA4 = ROC_30.rolling(15).mean()
        data_frame[i]['KST'] = RCMA1 + RCMA2*2 + RCMA3*3 + RCMA4*4

In [ ]:
#mass index
def Mass_index(data_frame, window_size = 9):

    for i,df in enumerate(data_frame):
        difference = df.High - df.Low
        ema9 = difference.ewm(span = window_size, adjust = False).mean()
        ema9_of = ema9.ewm(span= window_size, adjust = False).mean()
        mass = ema9 / ema9_of
        data_frame[i]['MassIndex'] = mass.rolling(25).sum()

In [ ]:
#True Strength Index(TSI)
def True_strength_index(data_frame, short_window=13, long_window=25):
    
    for i, df in enumerate(data_frame):
        close_diff = df.Close - df.Close.shift(1)
        pcs = close_diff.ewm(span = long_window, adjust = False).mean()
        pcds = pcs.ewm(span = short_window, adjust = False).mean()
        abs_close_diff = abs(close_diff)
        apcs = abs_close_diff.ewm(span = long_window, adjust = False).mean()
        apcds = apcs.ewm(span = short_window, adjust = False).mean()
        data_frame[i]['TSI'] = pcds/apcds * 100

In [ ]:
#The average directional movement index
#typicall 14 days
#append Positive, Negative direction indicator and Average True range
def ADX(data_frames, window_size = 14):

    for i, df in enumerate(data_frames):
        high = df.High
        low = df.Low
        close = df.Close
        true_range = pd.DataFrame(np.zeros((len(df), 3)), columns=['H-L', 'H-Cp', 'L-Cp'])
        true_range['H-L'] = (high - low)
        true_range['H-Cp'] = abs(high - close.shift(1))
        true_range['L-Cp'] = abs(low - close.shift(1))
        true_range = true_range.max(axis=1)
        average_true_range = true_range.rolling(window_size).mean()
        pos_direc_mv = (high - high.shift(1))
        neg_direc_mv = (low - low.shift(1))
        pos_direc_mv[pos_direc_mv < 0] = 0
        neg_direc_mv[neg_direc_mv > 0] = 0
        pos_direc_indi = 100 * (pos_direc_mv.ewm(alpha = 1/window_size).mean() / average_true_range)
        neg_direc_indi = abs(100 * (neg_direc_mv.ewm(alpha = 1/window_size).mean() / average_true_range))
        dx = (abs(pos_direc_indi - neg_direc_indi) / abs(pos_direc_indi + neg_direc_indi)) * 100
        adx = ((dx.shift(1) * (window_size - 1)) + dx) / window_size
        adx_smooth = adx.ewm(alpha = 1/window_size).mean()
        data_frames[i]['TR'] = true_range
        data_frames[i]['ATR'] = average_true_range
        data_frames[i]['Pos_DI'] = pos_direc_indi
        data_frames[i]['Neg_dI'] = neg_direc_indi
        data_frames[i][str(window_size) + '-day ADX'] = adx_smooth

In [ ]:
#Ultimate Oscillator
def ultimate_oscillator(data_frame):

    for i, df in enumerate(data_frame):
        buying_pressure = df.Close - np.minimum(df.Low, df.Close.shift(1))
        true_range = df.TR
        average7 = buying_pressure.rolling(7).sum() / true_range.rolling(7).sum()
        average14 = buying_pressure.rolling(14).sum() / true_range.rolling(14).sum()
        average28 = buying_pressure.rolling(28).sum() / true_range.rolling(28).sum()
        uo = (((average7*4) + (average14*2) + average28) / (4+2+1)) * 100
        data_frame[i]['UO'] = uo


In [ ]:
#Vortext indicator consists of plus and minus (VIN_pos and VIN_neg)
#VM+ and VM- are uptrend and downtrend movement respectively 
def vortex_indicator(data_frame, window_size = 14):

    for i, df in enumerate(data_frame):
        true_range = df.TR
        data_frame[i]['VM_plus'] = abs(df.High - df.Low.shift(1))
        data_frame[i]['VM_minus'] = abs(df.Low - df.High.shift(1))
        sum_tr = true_range.rolling(window_size).sum()
        sum_vm_plus = df.VM_plus.rolling(window_size).sum()
        sum_vm_minus = df.VM_minus.rolling(window_size).sum()
        data_frame[i]['VIN_pos'] = sum_vm_plus/sum_tr
        data_frame[i]['VIN_neg'] = sum_vm_minus/sum_tr

In [ ]:
#Percentage price oscillator
def ppo(data_frame, short_window = 12 , long_window = 26):

    for i,df in enumerate(data_frame):
        short_ma = df.Close.ewm(span = short_window, adjust = False).mean()
        long_ma = df.Close.ewm(span = long_window, adjust = False).mean()
        ppo = 100 * (short_ma - long_ma) / long_ma
        data_frame[i]['PPO'] = ppo

In [ ]:
#StochRSI 
#reading above 0.8 overbought , 0.2 oversold
def stochrsi(data_frame):

    for i,df in enumerate(data_frame):
        data_frame[i]['StochRSI'] = (df.RSI - df.RSI.rolling(14).min()) / (df.RSI.rolling(14).max() - df.RSI.rolling(14).min())

In [ ]:
#Envelope(ENV)
def Envelope(data_frame, window_size = 50):

    for i,df in enumerate(data_frame):
        sma = df.Close.rolling(window_size).mean()
        data_frame[i]['ENV_up_bound'] = sma + sma * 0.05
        data_frame[i]['ENV_low_bound'] = sma - sma * 0.05
        data_frame[i]['ENV_midpoint'] = sma

In [ ]:
#Chande momentum oscillaotr
def chande_momentum_oscillator(data_frame, window_size=14):

    for i,df in enumerate(data_frame):
        diff = df.Close.diff()
        up, down = diff.copy(), diff.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        higher_sum = up.rolling(window_size).sum()
        lower_sum = down.abs().rolling(window_size).sum()
        data_frame[i]['cmo'] = ((higher_sum - lower_sum) / (higher_sum + lower_sum)) * 100

In [ ]:
#Kaufman Adaptive Moving Average
def kama(data_frame, window_size=10, fast=2, slow=30):
    for i, df in enumerate(data_frame):
        volatility = abs(df['Close'] - df['Close'].shift(1)).rolling(window_size).sum()
        change = abs(df['Close'] - df['Close'].shift(window_size))
        er = change / volatility
        sc = (er * (fast / (fast + 1) - slow / (slow + 1)) + slow / (slow + 1)) ** 2
        kama = np.zeros(len(df))
        isfirst = True
        for j in range(len(df)):
            if np.isnan(sc[j]):
                kama[j] = np.nan
            else:
                if isfirst:
                    kama[j] = df['Close'][j]
                    isfirst = False
                else:
                    kama[j] = kama[j-1] + sc[j] * (df['Close'][j] - kama[j-1])
        data_frame[i]['Kama'] = kama

In [ ]:
def UlcerIndex(data_frame, window_size=14):
    """
    Calculates the Ulcer Index for a given data frame.

    Args:
    data_frame: Pandas DataFrame containing OHLCV data.
    window_size: The number of periods used to calculate the Ulcer Index.

    Returns:
    A Pandas DataFrame containing the Ulcer Index values.
    """
    for i, df in enumerate(data_frame):
      max_close = df['Close'].rolling(window=window_size).max()
      percent_drawdown = (df['Close'] / max_close - 1) * 100
      squared_drawdown = np.square(percent_drawdown)
      ulcer_index = np.sqrt(squared_drawdown.rolling(window=window_size).mean())
      data_frame[i]['Ulcer Index'] = ulcer_index


In [ ]:
#Klinger_Oscillator
def Klinger_Oscillator(data_frame, short_term=34, long_term=55, signal_period=13):
    
    for i, df in enumerate(data_frame):
      # Calculate the trend of money flow volume (MFV)
      trend_MFV = df.MF_Volume.rolling(window=short_term).sum() - df.MF_Volume.rolling(window=long_term).sum()
      
      # Calculate the trend of the volume force (VF)
      volume_force = df['Volume'] * ((2 * df['Close']) - df['High'] - df['Low']) / (df['High'] - df['Low'])
      trend_VF = volume_force.rolling(window=short_term).sum() - volume_force.rolling(window=long_term).sum()
      # Calculate the oscillator
      oscillator = trend_MFV - trend_VF
      # Calculate the signal line
      signal = oscillator.rolling(window=signal_period).mean()
      data_frame[i]['Klinger_Oscillator'] = oscillator
      
  

In [ ]:
#Mcclellan_Osciilator
def McClellan_Oscillator(data_frame, short_period=19, long_period=39):

    for i, df in enumerate(data_frame):
      ema_short = df.Close.ewm(span=short_period, adjust=False).mean()
      ema_long = df.Close.ewm(span=long_period, adjust=False).mean()

      oscillator = ema_short - ema_long

      ema_signal = oscillator.ewm(span=9, adjust=False).mean()
      data_frame[i]['McClellan_Osci'] = oscillator - ema_signal

In [ ]:
def choppiness_index(data_frame, window_size = 14):

    for i, df in enumerate(data_frame):
      high = df['High']
      low = df['Low']
      close = df['Close']

      atr = df.ATR
      tr = high - low

      # Calculate the True Range High (TRH) and True Range Low (TRL) for the lookback period
      trh = high.rolling(window=window_size).max()
      trl = low.rolling(window=window_size).min()

      # Calculate the absolute difference between the current close and the TRH
      diff_high = abs(close - trh)

      # Calculate the absolute difference between the current close and the TRL
      diff_low = abs(close - trl)

      # Calculate the Sum of TR over the lookback period
      sum_tr = tr.rolling(window=window_size).sum()

      # Calculate the Sum of the Absolute Differences over the lookback period
      sum_diff = diff_high.rolling(window=window_size).sum() + diff_low.rolling(window=window_size).sum()

      # Calculate the Raw Choppiness Index
      data_frame[i]['raw_ci'] = 100 * np.log10(sum_tr / sum_diff)

      # Smooth the Raw Choppiness Index with a 3 period EMA
      data_frame[i]['Ci'] = df.raw_ci.ewm(span = 3, adjust = False).mean()

       


In [ ]:
#elder_ray_index
def elder_ray_index(data_frame, period = 13):
    
    for i, df in enumerate(data_frame):
      ema = df['Close'].ewm(span=period, adjust=False).mean()
      data_frame[i]['bull_power'] = df['High'] - ema
      data_frame[i]['bear_power'] = df['Low'] - ema
      


In [ ]:
def SEB(data_frame, window_size=20, num_std=2):

    for i, df in enumerate(data_frame):
      se = df.Close.rolling(window_size).std() / np.sqrt(window_size)
      data_frame[i]['SEB_upper'] = df.Close.rolling(window_size).mean() + num_std * se
      data_frame[i]['SEB_lower'] = df.Close.rolling(window_size).mean() - num_std * se


#Task3

#Main Execution Part

#Task1

In [ ]:

def daily_percentage_changes(stock_data):
    close_prices = stock_data['Close']
    shifted_close_prices = close_prices.shift(-1)[:-1] # Shift the close prices by -1 (upwards)
    today_close_prices = close_prices[:-1] # Remove the last value from close_prices
    percentage_changes = (shifted_close_prices - today_close_prices) / today_close_prices
    stock_data['Close_pct_change'] = percentage_changes
    return percentage_changes


percentage_changes = [daily_percentage_changes(stock) for stock in data_frame]
flattened_changes = sorted(np.concatenate(percentage_changes))

total_data_points = len(flattened_changes)
level_1_threshold = np.nanpercentile(flattened_changes, 33.3)
level_2_threshold = np.nanpercentile(flattened_changes, 66.6)


def assign_level(percentage_change, level_1_threshold, level_2_threshold):
    if percentage_change < level_1_threshold:
        return 0  # Decrease
    elif percentage_change < level_2_threshold:
        return 1  # No big change
    else:
        return 2  # Increase

assigned_levels = [assign_level(change, level_1_threshold, level_2_threshold) for change in flattened_changes]
level_counts = np.bincount(assigned_levels)

print("Level 1 Threshold:", level_1_threshold)
print("Level 2 Threshold:", level_2_threshold)
print("Number of data points for each level:", level_counts)

#Task1
closed_percentage = daily_percentage_change_for_close(data)
thresholds = find_thresholds(closed_percentage)
levels = divide_into_levels(closed_percentage, thresholds)
threshold_number_of_data_points(levels, thresholds)
labels = [0, 1, 2]
label_vectors = create_label_vectors(levels, labels)



Level 1 Threshold: -0.002114146149282713
Level 2 Threshold: 0.0019722866658953677
Number of data points for each level: [1465866 1465866 1470268]
Threshold 1: -0.21141461492827127
Threshold 2: 0.19722866658953678
Number of increases: 1470268
Number of decreases: 1465866
Number of no big changes: 1465866


In [ ]:
#print(data_frame[0].iloc[1:101])
#print(label_vectors[0][101:102])

In [ ]:
print(data_frame[0].tail(5))

          Open      High       Low     Close    Volume  Close_pct_change
2197  0.680849  0.683633  0.672997  0.674055  0.351048          0.000000
2198  0.674389  0.674890  0.671326  0.674055  0.143069          0.002892
2199  0.674110  0.676227  0.672440  0.676004  0.312048         -0.009885
2200  0.675725  0.675837  0.668820  0.669321  0.279230         -0.007655
2201  0.669293  0.671326  0.662973  0.664198  0.552933               NaN


#Task2 functions

In [ ]:
#Taks2
#Indicators
daily_return(data_frame)
daily_log_return(data_frame)
standard_deviation(data_frame,5)
standard_deviation(data_frame,20)
median_price(data_frame)
volatility(data_frame)
ROC(data_frame, 5)
ROC(data_frame, 10)
ROC(data_frame, 14)
ROC(data_frame, 20)
RSI(data_frame)


In [ ]:
#Moving average 20, 50 ,and 200 periods
moving_average(data_frame, 5)
moving_average(data_frame, 10)
moving_average(data_frame, 20)
moving_average(data_frame, 50)


In [ ]:
#short term Exponential Moving average(EMA)
ema(data_frame, 12)
ema(data_frame, 26)
#long term EMA
ema(data_frame, 50)
ema(data_frame, 100) #chagne the period


In [ ]:
WMA(data_frame)
bollinger_bands(data_frame, 20)
bollinger_bands(data_frame, 50)

In [ ]:
momentum(data_frame)

In [ ]:
True_strength_index(data_frame)

In [ ]:
ADX(data_frame)

In [ ]:
MACD(data_frame)

In [ ]:
awesome_Oscillator(data_frame)

In [ ]:
stochastic_oscillator(data_frame)

In [ ]:
williams_R(data_frame)
CCI(data_frame)
#coppock_curve(data_frame) may be not appropriate for this project
on_balance_volume(data_frame)
ADTV(data_frame)
Fibonacci_retracement_levels(data_frame)
money_flow_index(data_frame)

In [ ]:
TEMA(data_frame)
VPT(data_frame)
pivot_points(data_frame)
VWAP(data_frame)
ichimoku_cloud(data_frame)
disparity(data_frame)
DPO(data_frame)
donchian_channel(data_frame)
EMV(data_frame)
vortex_indicator(data_frame)
ultimate_oscillator(data_frame)
chaikin_money_flow(data_frame)
accumulation_distribution(data_frame)
ForceIndex(data_frame)
Keltner_channel(data_frame)
Know_sure_thing(data_frame)
Mass_index(data_frame)
ppo(data_frame) 
stochrsi(data_frame)
Envelope(data_frame)
chande_momentum_oscillator(data_frame)
kama(data_frame)
Volume_Oscillator(data_frame)
zero_lag_ema(data_frame)
Hull_MA(data_frame)


In [ ]:
#aroon_indicator(data_frame)

In [ ]:
UlcerIndex(data_frame)

In [ ]:
Klinger_Oscillator(data_frame)

In [ ]:
McClellan_Oscillator(data_frame)

In [ ]:
choppiness_index(data_frame)

In [ ]:
elder_ray_index(data_frame)

In [ ]:
SEB(data_frame)

In [ ]:
#replace all the NaN to 0
for i in range(len(data_frame)):
    data_frame[i].fillna(0, inplace = True)

#display all the columns
pd.set_option('display.max_columns', None) 
print(len(data_frame[0].columns))
print(data_frame[0].tail(5))

105
          Open      High       Low     Close    Volume  Close_pct_change  \
2197  0.680849  0.683633  0.672997  0.674055  0.351048          0.000000   
2198  0.674389  0.674890  0.671326  0.674055  0.143069          0.002892   
2199  0.674110  0.676227  0.672440  0.676004  0.312048         -0.009885   
2200  0.675725  0.675837  0.668820  0.669321  0.279230         -0.007655   
2201  0.669293  0.671326  0.662973  0.664198  0.552933          0.000000   

      Daily_return  log_return  5-day S.D  20-day S.D    median  volatility  \
2197     -0.006794   -0.010192   0.003615    0.008223  0.678315    0.039817   
2198     -0.000334    0.000000   0.003862    0.008374  0.673108    0.035028   
2199      0.001893    0.002887   0.003624    0.008290  0.674333    0.036086   
2200     -0.006404   -0.009935   0.004198    0.007898  0.672328    0.040820   
2201     -0.005096   -0.007684   0.004780    0.007668  0.667149    0.033664   

      5-day ROC  10-day ROC  14-day ROC  20-day ROC        RSI  

#Train dataset

#Task3


In [ ]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

def normalize_features(stock_data):
    # Replace infinite values with NaN
    stock_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Handle NaN values (either fill them with appropriate values or drop rows/columns)
    # Option 1: Fill NaN values with the mean of the respective columns
    stock_data.fillna(0, inplace=True)
    
    # Option 2: Drop rows containing NaN values (uncomment the line below if you want to use this option)
    # stock_data.dropna(inplace=True)
    
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(stock_data)
    return pd.DataFrame(normalized_data, columns=stock_data.columns)


normalized_stock_data = [normalize_features(stock) for stock in data_frame]

In [ ]:
def split_data(training_data, train_ratio=0.8):
    split_index = int(len(training_data) * train_ratio)
    train_data = training_data[:split_index]
    val_data = training_data[split_index:]
    return train_data, val_data

train_data_X, validation_data_X = split_data(normalized_stock_data) #oringal data frame with features added so 2000*2202*102
train_data_Y, validation_data_Y = split_data(label_vectors) #3d vectors consisted of 3 features(decrease, no big change, increase) 2000*2201*3

train_data_X = np.array(train_data_X)
validation_data_X = np.array(validation_data_X)
train_data_Y = np.array(train_data_Y)
validation_data_Y = np.array(validation_data_Y)

#add (0,0,0) to match day since it has 2202
padded_train_data_Y = np.pad(train_data_Y, ((0, 0), (0, 1), (0, 0)), mode='constant', constant_values=0)
padded_validation_data_Y = np.pad(validation_data_Y, ((0, 0), (0, 1), (0, 0)), mode='constant', constant_values=0)

In [ ]:
print("train_data_X shape:", train_data_X.shape)
print("validation_data_X shape:", validation_data_X.shape)
print("padded_train_data_Y shape:", padded_train_data_Y.shape)
print("padded_validation_data_Y shape:", padded_validation_data_Y.shape)

train_data_X shape: (1600, 2202, 105)
validation_data_X shape: (400, 2202, 105)
padded_train_data_Y shape: (1600, 2202, 3)
padded_validation_data_Y shape: (400, 2202, 3)


In [ ]:
"feature selection for task4"

from scipy.stats import pearsonr

def select_features(data_X, data_Y, k=75):
    num_features = data_X.shape[2]
    correlations = np.zeros(num_features)

    target = data_Y[:, :, 2].flatten()

    for i in range(num_features):
        feature = data_X[:, :, i].flatten()

        if np.std(feature) == 0 or np.std(target) == 0:
            continue

        corr, _ = pearsonr(feature, target)
        correlations[i] = abs(corr)

    top_k_indices = np.argsort(correlations)[-k:][::-1]
    selected_data_X = data_X[:, :, top_k_indices]
    
    # Return top feature indices and their correlations as well
    top_k_correlations = correlations[top_k_indices]
    
    return selected_data_X, top_k_indices, top_k_correlations


In [ ]:

import torch.utils.data as data

class StockDataset(data.Dataset):
    def __init__(self, data_X, data_Y, window_size=100):
        self.data_X = data_X
        self.data_Y = data_Y
        self.window_size = window_size
        self.num_stocks, self.num_days, self.num_features = data_X.shape

    def __len__(self):
        return self.num_stocks * (self.num_days - self.window_size)

    def __getitem__(self, idx):
        stock_idx = idx // (self.num_days - self.window_size)
        day_idx = idx % (self.num_days - self.window_size)

        features = self.data_X[stock_idx, day_idx : day_idx + self.window_size, :]
        label = self.data_Y[stock_idx, day_idx + self.window_size, :]

        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

window_size = 100

In [ ]:
#features, label = train_dataset[0]
#print(normalized_stock_data[0].iloc[0:100]) #print 0~99 trading records 100
#print(label_vectors[0][100:101]) #print 101st label

#print(len(train_dataset))
#print(features.shape)
#print(label.shape)

#print(len(features))
#print(label)

#print(len(train_dataset))

#2000*2102*0.8

#print(len(train_dataset))
#print(len(validation_dataset))

In [ ]:
#LSTM uses 2d vector
class StockPredictionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(StockPredictionModel, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
        
        
    def forward(self, x):
        # LSTM layer
        out, _ = self.lstm(x)
        
        # Get the output from the last time step of the LSTM layer
        out = out[:, -1, :]
        
        # Fully connected layers
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

# Hyperparameters
input_size = train_data_X.shape[2] #after feature selection. If we do no apply feature selection, it should be just train_x, right now we didn't apply feature selection
hidden_size = 512
num_layers = 3
num_classes = train_data_Y.shape[2]

model = StockPredictionModel(input_size, hidden_size, num_layers, num_classes)


In [ ]:
train_loaders = []
stock_batch_size = 160
num_days = 500

for i in range(0, len(train_data_X), stock_batch_size): #load 160 stock
    temp_train_data = []
    temp_label_data = []
    for stock, label in zip(train_data_X[i:i+stock_batch_size], padded_train_data_Y[i:i+stock_batch_size]):
        temp_train_data.append(stock[:num_days])
        temp_label_data.append(label[:num_days])
    
    # Convert lists to numpy arrays
    temp_train_data = np.array(temp_train_data)
    temp_label_data = np.array(temp_label_data)
    
    train_dataset = StockDataset(temp_train_data, temp_label_data, window_size=100)
    train_dataloader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    train_loaders.append(train_dataloader)

validation_loaders = []
for i in range(0, len(validation_data_X), 40): #load 40 stocks
    validation_dataset = StockDataset(validation_data_X[i:i+40], padded_validation_data_Y[i:i+40], window_size=100)
    validation_dataloader = DataLoader(validation_dataset, batch_size=1024, shuffle=True)
    validation_loaders.append(validation_dataloader)


In [ ]:
# Check if a GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    true_positive = 0
    positive_predictions = 0
    
    with torch.no_grad():
        for features, labels in dataloader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            _, true_labels = torch.max(labels.data, 1)
            
            total += true_labels.size(0)
            correct += (predicted == true_labels).sum().item()
            positive_predictions += (predicted == 2).sum().item()
            true_positive += ((predicted == 2) & (true_labels == 2)).sum().item()
    
    accuracy = correct / total
    if positive_predictions > 0:
      precision = true_positive / positive_predictions
    else:
      precision = 0
    positive_prediction_percentage = positive_predictions / total

    return accuracy, precision, positive_prediction_percentage

best_accuracy = 0.0 # initialize the best accuracy to 0
criterion = nn.CrossEntropyLoss()
# Train the model
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(num_epochs):
    print(f"Starting epoch {epoch + 1} of {num_epochs}")
    mini_epoch = 0
    for loaders in zip(train_loaders, validation_loaders):
        train_dataloader, validation_dataloader = loaders
        model.train() # set the model to training mode


        for i, (features, labels) in enumerate(train_dataloader):
            #print(f"Number of batches {i + 1}: {len(train_dataloader)}")
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, torch.max(labels, 1)[1].long())
            loss.backward()
            optimizer.step()
        
        mini_epoch += 1
        # Evaluate the model on training data
        print(f"mini_epoch [{mini_epoch}/{10}]")
        train_accuracy, train_precision, train_positive_prediction_percentage = evaluate_model(model, train_dataloader)
        print(f"Training Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Positive Predictions: {train_positive_prediction_percentage:.4f}")

        # Evaluate the model on validation data
        validation_accuracy, validation_precision, validation_positive_prediction_percentage = evaluate_model(model, validation_dataloader)
        print(f"Validation Accuracy: {validation_accuracy:.4f}, Precision: {validation_precision:.4f}, Positive Predictions: {validation_positive_prediction_percentage:.4f}")

        # Check if the validation accuracy has improved
        if validation_accuracy > best_accuracy:
            best_accuracy = validation_accuracy
            torch.save(model.state_dict(), 'best_model.pth') # save the model weights

print(f"Best validation accuracy: {best_accuracy:.4f}")

#The model is trained for num_epochs epochs, with each epoch consisting of 40 mini-epochs. In each mini-epoch, the model is trained on a batch of 1024 samples, 
# and the training and validation accuracies, precisions, and percentages of positive predictions are printed.
# A training epoch can be divided into multiple mini-epochs, each consisting of multiple batches of training data. The term "mini-epoch" is not a standard term in machine learning,
# but it is a useful way to divide the training process into smaller parts, especially when dealing with large datasets or models that take a long time to train

Starting epoch 1 of 1
mini_epoch [1/10]
Training Accuracy: 0.3976, Precision: 0.3031, Positive Predictions: 0.0435
Validation Accuracy: 0.3683, Precision: 0.2449, Positive Predictions: 0.0445
mini_epoch [2/10]
Training Accuracy: 0.4194, Precision: 0.3459, Positive Predictions: 0.2041
Validation Accuracy: 0.3960, Precision: 0.2923, Positive Predictions: 0.2129
mini_epoch [3/10]
Training Accuracy: 0.4429, Precision: 0.3694, Positive Predictions: 0.3661
Validation Accuracy: 0.4143, Precision: 0.3518, Positive Predictions: 0.3157
mini_epoch [4/10]
Training Accuracy: 0.4135, Precision: 0.3719, Positive Predictions: 0.3543
Validation Accuracy: 0.4010, Precision: 0.3424, Positive Predictions: 0.2842
mini_epoch [5/10]
Training Accuracy: 0.4693, Precision: 0.3853, Positive Predictions: 0.2852
Validation Accuracy: 0.3956, Precision: 0.3773, Positive Predictions: 0.3901
mini_epoch [6/10]
Training Accuracy: 0.4865, Precision: 0.3860, Positive Predictions: 0.3603
Validation Accuracy: 0.4120, Precis

#Task5

In [ ]:
num_models = 10
models = [StockPredictionModel(input_size, hidden_size, num_layers, num_classes) for _ in range(num_models)]

for i, model in enumerate(models):
    print(f"Training model {i + 1} of {num_models}")
    model.to(device)
    
    # Train the model
    num_epochs = 10
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch + 1} of {num_epochs}")
        mini_epoch = 0
        
        for loaders in zip(train_loaders, validation_loaders):
            train_dataloader, validation_dataloader = loaders
            model.train()  # set the model to training mode

            for _, (features, labels) in enumerate(train_dataloader):
                features, labels = features.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(features)
                loss = criterion(outputs, torch.max(labels, 1)[1].long())
                loss.backward()
                optimizer.step()

            mini_epoch += 1
            print(f"mini_epoch [{mini_epoch}/{10}]")

            # Evaluate the model on training data
            train_accuracy, train_precision, train_positive_prediction_percentage = evaluate_model(model, train_dataloader)
            print(f"Training Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Positive Predictions: {train_positive_prediction_percentage:.4f}")

            # Evaluate the model on validation data
            validation_accuracy, validation_precision, validation_positive_prediction_percentage = evaluate_model(model, validation_dataloader)
            print(f"Validation Accuracy: {validation_accuracy:.4f}, Precision: {validation_precision:.4f}, Positive Predictions: {validation_positive_prediction_percentage:.4f}")

    # Save the model weights
    torch.save(model.state_dict(), f'model_{i}.pth')


Training model 1 of 10
Starting epoch 1 of 1
mini_epoch [1/10]
Training Accuracy: 0.4128, Precision: 0.3732, Positive Predictions: 0.1402
Validation Accuracy: 0.4221, Precision: 0.3144, Positive Predictions: 0.1022
mini_epoch [2/10]
Training Accuracy: 0.4221, Precision: 0.3514, Positive Predictions: 0.1549
Validation Accuracy: 0.4105, Precision: 0.3016, Positive Predictions: 0.1617
mini_epoch [3/10]
Training Accuracy: 0.4425, Precision: 0.3693, Positive Predictions: 0.3634
Validation Accuracy: 0.4142, Precision: 0.3540, Positive Predictions: 0.3297
mini_epoch [4/10]
Training Accuracy: 0.4087, Precision: 0.3564, Positive Predictions: 0.3925
Validation Accuracy: 0.3879, Precision: 0.3266, Positive Predictions: 0.4101
mini_epoch [5/10]
Training Accuracy: 0.4676, Precision: 0.3677, Positive Predictions: 0.2569
Validation Accuracy: 0.4013, Precision: 0.3691, Positive Predictions: 0.3504
mini_epoch [6/10]
Training Accuracy: 0.4816, Precision: 0.3865, Positive Predictions: 0.2087
Validation A

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Validation Accuracy: 0.4319, Precision: 0.3798, Positive Predictions: 0.4115
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-99-3e437d0ee47b>", line 40, in <module>
    torch.save(model.state_dict(), f'model_{i}.pth')
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 422, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 309, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 287, in __init__
    super(_open_zipfile_writer_file, self).__init__(torch._C.PyTorchFileWriter(str(name)))
RuntimeError: File model_9.pth cannot be opened.

During handling of the above exception, another exception occurred:

Traceback (most recent c

RuntimeError: ignored

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score

from sklearn.metrics import accuracy_score, precision_score
from sklearn.ensemble import AdaBoostClassifier

def voting(models, dataloader):
    all_preds = []
    
    for model in models:
        model.eval()
        preds = []
        
        with torch.no_grad():
            for features, _ in dataloader:
                features = features.to(device)
                outputs = model(features)
                _, predicted = torch.max(outputs.data, 1)
                preds.extend(predicted.cpu().numpy())
                
        all_preds.append(preds)
    
    return np.array(all_preds)

def blending(models, dataloader):
    all_outputs = []

    for model in models:
        model.eval()
        outputs_list = []

        with torch.no_grad():
            for features, _ in dataloader:
                features = features.to(device)
                outputs = model(features)
                outputs_list.extend(outputs.cpu().numpy())

        all_outputs.append(outputs_list)

    return np.mean(all_outputs, axis=0)

datasets = {'Training': train_dataloader, 'Validation': validation_dataloader}

for dataset_name, dataloader in datasets.items():
    print(f"Evaluating on {dataset_name} set:")
    # Voting
    voting_preds = voting(models, dataloader)
    voting_accuracy = accuracy_score(np.argmax(dataloader.dataset.data_Y, axis=1), np.argmax(np.mean(voting_preds, axis=0), axis=1))
    voting_precision = precision_score(np.argmax(dataloader.dataset.data_Y, axis=1), np.argmax(np.mean(voting_preds, axis=0), axis=1), average='weighted')
    
    print(f"Voting - Accuracy: {voting_accuracy:.4f}, Precision: {voting_precision:.4f}")



In [ ]:
# Blending
blending_outputs = blending(models, dataloader)
blending_accuracy = accuracy_score(np.argmax(dataloader.dataset.data_Y, axis=1), np.argmax(blending_outputs, axis=1))
blending_precision = precision_score(np.argmax(dataloader.dataset.data_Y, axis=1), np.argmax(blending_outputs, axis=1), average='weighted')

print(f"Blending - Accuracy: {blending_accuracy:.4f}, Precision: {blending_precision:.4f}")

# AdaBoost
X = np.vstack([features.numpy() for features, _ in dataloader])
y = np.argmax(np.vstack([labels.numpy() for _, labels in dataloader]), axis=1)

# Create a list of weak classifiers
weak_classifiers = []
for model in models:
    model.eval()
    weak_classifier = lambda x: torch.max(model(torch.tensor(x, dtype=torch.float32).to(device)).cpu(), 1)[1].numpy()
    weak_classifiers.append(weak_classifier)

# Train the AdaBoost classifier
adaboost = AdaBoostClassifier(base_estimator=None, n_estimators=len(weak_classifiers), algorithm='SAMME.R')
adaboost.fit(X, y, sample_weight=None)

# Evaluate the AdaBoost classifier
adaboost_preds = adaboost.predict(X)
adaboost_accuracy = accuracy_score(y, adaboost_preds)
adaboost_precision = precision_score(y, adaboost_preds, average='weighted')

print(f"AdaBoost - Accuracy: {adaboost_accuracy:.4f}, Precision: {adaboost_precision:.4f}")

#Task6

In [ ]:
# Create DataLoaders for the test sets
test_dataset_1 = StockDataset(test1['data_X'], test1['data_Y'], window_size=100)
test_dataloader_1 = DataLoader(test_dataset_1, batch_size=1024, shuffle=True)

test_dataset_2 = StockDataset(test2['data_X'], test2['data_Y'], window_size=100)
test_dataloader_2 = DataLoader(test_dataset_2, batch_size=1024, shuffle=True)

# Load the best model from Task 5
best_model = StockPredictionModel(input_size, hidden_size, num_layers, num_classes)
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.to(device)

# Evaluate the model on the test sets
datasets = {'Test Set 1': test_dataloader_1, 'Test Set 2': test_dataloader_2}

for dataset_name, dataloader in datasets.items():
    print(f"Evaluating on {dataset_name}:")
    accuracy, precision, positive_prediction_percentage = evaluate_model(best_model, dataloader)
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Positive Predictions: {positive_prediction_percentage:.4f}")
